In [1]:
import process_data

In [2]:
import csv
import torch
import numpy as np
from os.path import join

class Dataloader(object):
    def __init__(self, path, train_filename, test_filename):
        self.train_filepath = join(path, train_filename)
        self.test_filepath = join(path, test_filename)
    
    def read_images_labels(self, image_filepath):
        labels = []
        images = []

        with open(image_filepath, "r") as f:
            file = csv.reader(f)
            header = next(file)
            
            for lines in file:
                lines = [float(item) for item in lines]
                labels.append(lines[0])
                image = torch.tensor(lines[1:])
                images.append(image.view(1,28,28))
        
        # print(len(labels), len(images))
        if len(labels) != len(images):
            raise ValueError('Mismatched number of labels and images')

        return images, labels

    def load_data(self):
        train_data, train_labels = self.read_images_labels(self.train_filepath)
        test_data, test_labels = self.read_images_labels(self.test_filepath)
        return (train_data, train_labels), (test_data, test_labels)

In [3]:
file_path = r'A:\Alsou Bek\Documents\programs\Clothing_Classification_model\data'
train_filename = 'fashion-mnist_train.csv'
test_filename = 'fashion-mnist_test.csv'

dataloader = Dataloader(file_path, train_filename, test_filename)
(train_data, train_labels), (test_data, test_labels) = dataloader.load_data()

In [4]:
train_data[0].shape

torch.Size([1, 28, 28])

In [5]:
import process_to_dataset

In [6]:
train_dataset = process_to_dataset.MyDataset(train_data, train_labels)
test_dataset = process_to_dataset.MyDataset(test_data, test_labels)

In [7]:
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader

In [8]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [9]:
import models

In [10]:
net = models.ConvNN()

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    avg_loss = 0
    losses = []
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()

        # print statistics
        # running_loss += loss.item()
        # if (i+1) % 25 == 0:    # print every 2000 mini-batches
        #     print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 25:.3f}')
        #     running_loss = 0.0
            
    losses.append()

print('Finished Training')

Finished Training


In [13]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [14]:
import matplotlib as plt

In [15]:
dataiter = iter(train_dataloader)
images, labels = next(dataiter)

In [16]:
net = models.ConvNN()
net.load_state_dict(torch.load(PATH, weights_only=True))

<All keys matched successfully>

In [17]:
outputs = net(images)

In [18]:
correct = 0
total = 0

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 90.28 %


In [ ]:
categories = {0: 'T-shirt/top',
            1: 'Trouser',
            2: 'Pullover',
            3: 'Dress',
            4: 'Coat',
            5: 'Sandal',
            6: 'Shirt',
            7: 'Sneaker',
            8: 'Bag',
            9: 'Ankle boot'}

: 